<a href="https://colab.research.google.com/github/taikutsu91/Git-Treino/blob/master/BIRP_DOWNLOADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m pip install mutagen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 193 kB 5.1 MB/s 


In [2]:
!pip install humanfriendly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.4 MB/s 


In [3]:
import os
import shutil
from bs4 import BeautifulSoup
import requests
import time
import re
from urllib.parse import urljoin
from mutagen.mp3 import MP3
from humanfriendly import format_timespan

In [4]:
YEAR, MONTH = '2020', 'august'

In [6]:
DOWNLOAD_LINK = 'https://d1e5xmqmk0w5rl.cloudfront.net/playlists/'

def get_songs_url():
    URL = f'https://www.birp.fm/playlist/{YEAR}/{MONTH}-{YEAR}'
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    if r.ok:
      songs_url = soup.find_all("div", attrs={"class": "track-playbutton"})
      ok_urls = []
      bad_urls = []
      urls = []
      songs_not_clean = []
      for i in songs_url:
        link = urljoin(DOWNLOAD_LINK, i.get('id'))
        songs_not_clean.append(i.get('id'))
        urls.append(link)
        r = requests.get(link)
        if r.ok:
          ok_urls.append(link)
        else:
          bad_urls.append(link)
    return ok_urls, bad_urls, urls, songs_not_clean

def get_indexes(bad_songs, good_songs, urls):
    idx_bad = [urls.index(idx) for idx in bad_songs]
    idx_good = [urls.index(idx) for idx in good_songs]

    return idx_bad, idx_good

def idx_songs(idx_bad, idx_good, songs_list):
  good_song = [songs_list[idx] for idx in idx_good]
  bad_songs = [songs_list[idx] for idx in idx_bad]
  return good_song, bad_songs

def get_song_name(YEAR, MONTH):
    url = f"https://www.birp.fm/playlist/{YEAR}/{MONTH}-{YEAR}"
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        title = soup.find_all('span', attrs={'class':'song-title'})
        artist = soup.find_all('span', attrs={'class':'song-artist'})
        title = [x.text for x in title]
        artist = [x.text for x in artist]
        clean_artist= [re.sub(r'[^a-zA-Z0-9 ]', '', i) for i in artist]
        clean_title = [re.sub(r'[^a-zA-Z0-9 ]', '', i) for i in title]
        full_song = [f'{val+1:03} - {x} - {y}' for val, (x, y) in enumerate(zip(clean_artist, clean_title))]
        return full_song
    except Exception as e:
        print(e)

def creater_folder():
    folder_name = os.path.join('/content', f'Various Artists - BIRP! {MONTH.capitalize()} {YEAR}')
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    return os.path.abspath(folder_name)

def song_download(songs_url, songs_name):
    os.chdir(PATH)
    global URLS
    for url, song in zip(songs_url, songs_name):
        time.sleep(1)
        music = requests.get(url)
        with open(f"{song}.mp3", 'wb') as f:
            f.write(music.content)
        print(f'Downloading {song}...')
    print(f'Download {len(songs_url)}/{len(URLS)}')

def download_album_cover():
  os.chdir(PATH)
  img_data = requests.get(f'https://www.birp.fm/images/albumart/{MONTH}{YEAR}.jpg').content
  with open('albumcover.jpg', 'wb') as img:
    img.write(img_data)

def get_external_link():
    os.chdir(PATH)
    url = f"https://www.birp.fm/playlist/{YEAR}/{MONTH}-{YEAR}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    data = soup.findAll(class_='playlist-options')
    links = []
    nome = []
    for a in data[0].findAll(attrs={'class':"btn playlist-btn"}):
      links.append(a.get('href'))
      nome.append(a.text)
    for idx, i  in enumerate(links):
        if i.endswith('.torrent'):
          a =  urljoin('https://www.birp.fm', i.replace(' ', '%20'))
          links[idx] = a
    correto = [f"{n}: {l}" for  n, l in zip(nome, links)]
    correto = correto[1::]
    with open(f"External Links BIRP! {MONTH.capitalize()} {YEAR}.txt", 'w') as f:
      f.write(f'External Links Various Artists - BIRP! {MONTH.capitalize()} {YEAR}')
      f.write('\n')
      for line in correto:
        f.write(line+"\n")

def text_bad_songs(bad_songs, ok_songs):
    os.chdir(PATH)
    soma_sec = 0
    pct = (len(ok_songs)/(len(bad_songs) + len(ok_songs))) * 100
    for i in os.listdir():
        if i.endswith('mp3'):
            soma_sec+= MP3(i).info.length
    with open(f'BIRP! {MONTH.capitalize()} {YEAR}.txt', 'w') as f:
        f.write(f"BIRP! {MONTH.capitalize()} {YEAR}" + '\n' + '\n')
        f.write('Follow BIRP!' + '\n')
        f.write('YOUTUBE: https://www.youtube.com/channel/UC-HHJWCzskrsPdEjYulBg4w' + '\n')
        f.write('TWITTER: https://twitter.com/birp' + '\n')
        f.write('SPOTIFY: https://open.spotify.com/user/1217281510' + '\n')
        f.write('FACEBOOK: https://www.facebook.com/birp.fm' + '\n')
        f.write('SOUNDCLOUD: https://soundcloud.com/birp' + '\n')
        f.write('INSTAGRAM: https://www.instagram.com/birpfm/' + '\n' + '\n')
        f.write(f"Downloaed {len(ok_songs)} songs..." + '\n')
        f.write(f'{len(ok_songs)}/{(len(bad_songs) + len(ok_songs))}' + '\n')
        f.write(f'Downloaed {pct:.2f}%' + '\n')
        f.write(f'Album lenght is {format_timespan(soma_sec, max_units=2)}' + '\n')
        f.write(f"{len(bad_songs)} songs are not availiable to download" + '\n' + '\n')
        for line in bad_songs:
            f.write(''.join(line.split('/')[2::]))
            f.write('\n')

In [ ]:
shutil.make_archive(PATH, 'zip', PATH)

'/content/Various Artists - BIRP! August 2010.zip'

In [8]:
PATH

'/content/Various Artists - BIRP! August 2020'